In [106]:
# My Standard Spark Session!

# Python libraries:
import os
import sys
import re
from dateutil import parser
# import datetime
from datetime import datetime
from datetime import date
import builtins
import json
import functools
import operator
from itertools import product

# Numpy & Pandas!
import numpy as np
import pandas as pd
pd.options.display.float_format = '{:20,.2f}'.format
pd.options.display.max_columns = None

# Spark!
from pyspark import SparkContext
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql.window import *
from pyspark.sql import SparkSession, Row


spark = SparkSession.builder.appName("myapp").getOrCreate()

#     spark = SparkSession.builder.master("yarn")\
#     .config("spark.executor.instances", "32")\
#     .config("spark.executor.cores", "4")\
#     .config("spark.executor.memory", "4G")\
#     .config("spark.driver.memory", "4G")\
#     .config("spark.executor.memoryOverhead","4G")\
#     .config("spark.yarn.queue","Medium")\
#     .appName("myapp")\
#     .getOrCreate()

sc = spark.sparkContext
spark.conf.set("spark.sql.sources.partitionColumnTypeInference.enabled", "false")
spark.conf.set("spark.debug.maxToStringFields","true")



from pyspark.sql.functions import monotonically_increasing_id

In [2]:
%load_ext autoreload
%autoreload 2
# The autoreload extension is already loaded. To reload it, use:
#  %reload_ext autoreload


# mylib:
my_library = os.path.expanduser('~/.myconfigs')
my_spark = os.path.expanduser('~/spark2_dfanalysis')
sys.path.append(my_library)
sys.path.append(my_spark)


from shared.app_context import *
from builder.DataFrameBuild import *

ctx = ApplicationContext("Dev-Job")

DFB = DataFrameBuild(ctx.spark)

print("%16s  %s" % ("Python Version:",sys.version))
print("%16s  %s" % ("Python Path:",os.path.dirname(sys.executable)))
print("%16s  %s" % ("My Python Libs:",my_library))
print("%16s  %s" % ("My Spark Dir:",my_spark))
print("%16s  %s" % ("My Spark Ctx:",ctx.spark))
# print(ctx.spark)
# print(os.listdir(my_spark))
# print(sys.path)
# print("\n")

 Python Version:  3.6.8 |Anaconda, Inc.| (default, Feb 21 2019, 18:30:04) [MSC v.1916 64 bit (AMD64)]
    Python Path:  C:\ProgramData\Anaconda3\envs\py36
 My Python Libs:  C:\Users\MerzDal/.myconfigs
   My Spark Dir:  C:\Users\MerzDal/spark2_dfanalysis
   My Spark Ctx:  <pyspark.sql.session.SparkSession object at 0x0000018D73DC39B0>


# Example Dataframes:

In [3]:
num = 10000

In [4]:
# 2 columns: integer, string

lst_cars = [random.choice(['Honda','Toyota','Chevy','Ford','Tesla','Volkswagon','Hyundai','Jeep']) for x in range(num)]

df2 = DFB.arrays_to_dataframe(
    [[int(x) for x in np.linspace(1,num,num)],
    lst_cars],
    ['index','make'])

df2.limit(4).toPandas()

,index,make
0,1,Tesla
1,2,Tesla
2,3,Jeep
3,4,Honda


In [236]:
# 5 columns: strings, ints, float
import random

lst_months = ['January','Feburary','March','April','May','June',
              'July','August','September','October','November','December']

years = [random.randint(1900,2019) for x in range(num)]
# print(years)

ages = [(2019 - x) for x in years]
# print(ages)

months = [random.choice(lst_months) for x in range(num)]
# print(months)

worth = [random.random() * 100000 for x in range(num)]
# print(worth)

lst_names = ['sarah','bill','steve','mary','alyssa','brian','elizabeth','josh','ryan','katie','connor','erica','lisa','doug']
names = [random.choice(lst_names) for x in range(num)]

lst_states = ['OH','KY','IN','MI','PA']
states = [random.choice(lst_states) for x in range(num)]

lst_cars = ['Chevy','Honda','Toyota','Acura','Jeep']
cars = [random.choice(lst_cars) for x in range(num)]



# dfA = DFB.arrays_to_dataframe([names,months,years,ages,worth,states],
#                              ['name','month','year','age','net_worth','states'])

dfA = DFB.arrays_to_dataframe([names,months,ages,states,cars],
                             ['name','m','age','states','cars'])


dfA.limit(4).toPandas()

,name,m,age,states,cars
0,ryan,October,106,PA,Honda
1,josh,June,74,OH,Honda
2,sarah,Feburary,44,PA,Toyota
3,sarah,June,61,KY,Jeep


In [6]:
# 5 columns: index(int), strings, ints, double

df1 = DFB.arrays_to_dataframe(
    [[int(x) for x in np.linspace(1,num,num)],
     DFB.build_array("string",num=num,width=3),
     DFB.build_array("integer",num=num,nrange=(100,999)),
     DFB.build_array("double",num=num,nrange=(1000,100000))],
    ['index','vin','division','price'])

df1.limit(4).toPandas()

,index,vin,division,price
0,1,dwb,640,"57,901.90"
1,2,bgp,627,"71,917.83"
2,3,pbi,870,"16,384.60"
3,4,awg,587,"74,793.71"


In [34]:
# 10 columns: index(int), strings, ints, double

df1 = DFB.arrays_to_dataframe(
    [[int(x) for x in np.linspace(1,num,num)],
     DFB.build_array("string",num=num,width=3),
     DFB.build_array("integer",num=num,nrange=(100,999)),
     DFB.build_array("integer",num=num,nrange=(1,12)),
     DFB.build_array("integer",num=num,nrange=(1,28)),
     DFB.build_array("integer",num=num,nrange=(2010,2019)),
     DFB.build_array("integer",num=num,nrange=(0,23)),
     DFB.build_array("integer",num=num,nrange=(0,59)),
     DFB.build_array("integer",num=num,nrange=(0,59)),
     DFB.build_array("double",num=num,nrange=(1000,100000))],
    ['index','vin','division','month','day','year','hour','minute','second','price'])

df1.limit(4).toPandas()

,index,vin,division,month,day,year,hour,minute,second,price
0,1,fis,453,6,20,2011,9,44,16,"45,843.71"
1,2,bxb,622,6,12,2017,5,11,32,"18,180.77"
2,3,xkn,800,11,19,2013,22,35,20,"91,911.71"
3,4,bys,495,10,15,2017,10,57,1,"91,918.19"


# States ...

In [237]:
dfA.count()

10000

In [238]:
dfA.limit(10).toPandas()

,name,m,age,states,cars
0,ryan,October,106,PA,Honda
1,josh,June,74,OH,Honda
2,sarah,Feburary,44,PA,Toyota
3,sarah,June,61,KY,Jeep
4,bill,September,117,OH,Honda
5,josh,June,36,MI,Jeep
6,ryan,April,14,MI,Toyota
7,josh,Feburary,23,KY,Honda
8,steve,Feburary,32,IN,Honda
9,doug,Feburary,17,MI,Chevy


In [ ]:
df1 = dfA.limit(10)

In [ ]:
df1.limit(12).toPandas()

# index

In [ ]:
df1.count()

In [ ]:
cols = ["month", "age", "states"]

In [ ]:
def df_vstack(df, col1, col2):
    df1 = df.drop(col2)\
    .withColumn("index", monotonically_increasing_id())\
    .withColumnRenamed(col1, "att")\
    .withColumn("attr", col("att").cast(StringType()))\
    .select("index", "name", "attr")
    
    print(df1.printSchema())
    
    
    df2 = df.drop(col1)\
    .withColumn("index", monotonically_increasing_id())\
    .withColumnRenamed(col2, "att")\
    .withColumn("attr", col("att").cast(StringType()))\
    .select("index", "name", "attr")
    
    print(df2.printSchema())
    
    
    dfr = df1.join(df2, "index", "outer")
    return dfr

In [ ]:
df2 = df_vstack(df1, "month", "age")

In [ ]:
df2.columns

In [ ]:
df2.count()

In [ ]:
df2.limit(10).toPandas()

# "stack"

In [239]:
# df1b = df1.withColumn("age", col("age").cast(StringType())

In [240]:
# df_temp.select(*(col(c).cast("float").alias(c) for c in df_temp.columns))



#.where("Total is not null"))

In [246]:
# explicit stack example:
# df2 = df1.select(col("name"),
# expr("stack(2, 'month', month, 'states', states) as (Type, Attribute)"))\
# .drop("Type")


# stack function
def full_stacker(df, col1, stacked_cols):
    #print(col1)
    #print(stacked_cols)
    
    col_expr = ",".join(["'{}', {}".format(x,x) for x in stacked_cols])
    print(col_expr)
#     col_expr = " ".join(col_expr)
#     print(col_expr, len(col_expr))
#     return
    
    #                  len   0    0    1    1     drop Attribute
    full_expr = "stack({}, {}) as ({})".format(
        len(stacked_cols), col_expr, "Attribute")

    
    print("full:", full_expr)
    
    df1 = df.select(col(col1),
    expr(full_expr))\
    .drop("dropped")
    
    #print(df1.columns)
    #.drop("Type")

    return df1
    

df2 = full_stacker(dfA, "name", ["m", "states", "cars"])

# df3 = full_stacker(df2, "name", ["Attribute", "states"])

print(df2.count())

df2.limit(10).toPandas()

'm', m,'states', states,'cars', cars
full: stack(3, 'm', m,'states', states,'cars', cars) as (dropped, Attribute)
30000


,name,Attribute
0,ryan,October
1,ryan,PA
2,ryan,Honda
3,josh,June
4,josh,OH
5,josh,Honda
6,sarah,Feburary
7,sarah,PA
8,sarah,Toyota
9,sarah,June


# other

In [64]:
df1a = df1.select("name","states")

In [65]:
df1b = df1.select("name","age")

In [66]:
df1c = df1.select("name","month")

In [73]:
dfpa = df1a.groupby("name").pivot("states").count()

In [75]:
dfpb = df1b.groupby("name").pivot("age").count()

In [76]:
dfpc = df1c.groupby("name").pivot("month").count()

In [77]:
dfpa.limit(4).toPandas()

,name,IN,KY,MI,PA
0,mary,2.00,nan,None,None
1,elizabeth,1.00,nan,None,None
2,ryan,nan,1.00,None,None
3,connor,nan,2.00,None,None


In [ ]:
# UDF

# UDF's with lambdas!


# functions:
def leading_zeroes(x,width=2):
    return str(x).zfill(width)

def nohyphens(z):
    return re.sub("-","",str(z))

def date_from_str_mdY(z):
    return datetime.strptime(z,'%m-%d-%Y')

def datetime_from_str_mdY_time(z):
    return datetime.strptime(z,'%m-%d-%Y %H:%M:%S')
    # dt = datetime.strptime("21/11/06 16:30", "%d/%m/%y %H:%M")
    
    
def colval(x):
    return x


# udf's:
udf_leading_zeroes2 = udf(lambda z: leading_zeroes(z,2),StringType())
udf_leading_zeroes4 = udf(lambda z: leading_zeroes(z,4),StringType())
udf_leading_zeroes6 = udf(lambda z: leading_zeroes(z,6),StringType())

# NB: the modified but dual use of "nohyphens"
udf_nohyphen_str = udf(lambda z: nohyphens(z),StringType())
udf_nohyphen_int = udf(lambda z: int(nohyphens(z)),IntegerType())

# date, datetime:
udf_date_from_str_mdY = udf(lambda z: date_from_str_mdY(z),DateType())
udf_datetime_from_str_mdY_time = udf(lambda z: datetime_from_str_mdY_time(z),TimestampType())




In [ ]:
dfpb.limit()

In [98]:
df3 = df1\
.withColumn("a1", lit(col("states")))\
.withColumn("a2", lit(col("age")))\
.withColumn("a3", lit(col("month")))

In [99]:
df3a = df3.select("name", "a1")
df3b = df3.select("name", "a2")\
.withColumnRenamed("a2", "a1")

In [100]:
df4 = df3a.alias("a").join(df3b.alias("b"), col("a.name") == col("b.name"), "left_outer")

In [101]:
df4.count()
# 714k

14

In [102]:
df4.limit(20).toPandas()

,name,a1,name,a1
0,mary,IN,mary,65
1,mary,IN,mary,9
2,elizabeth,IN,elizabeth,20
3,ryan,KY,ryan,35
4,connor,KY,connor,62
5,connor,KY,connor,68
6,bill,MI,bill,25
7,lisa,PA,lisa,4
8,connor,KY,connor,62
9,connor,KY,connor,68


In [103]:
df3.count()

10

In [104]:
df3.limit(30).toPandas()

,name,month,age,states,a1,a2,a3
0,mary,April,9,IN,IN,9,April
1,elizabeth,Feburary,20,IN,IN,20,Feburary
2,ryan,November,35,KY,KY,35,November
3,connor,June,68,KY,KY,68,June
4,bill,Feburary,25,MI,MI,25,Feburary
5,lisa,October,4,PA,PA,4,October
6,connor,Feburary,62,KY,KY,62,Feburary
7,steve,January,32,KY,KY,32,January
8,doug,January,94,MI,MI,94,January
9,mary,November,65,IN,IN,65,November


In [67]:
# df_index = df1.join(df2,df1.index == df2.index)

df2 = df1a.alias("a")\
.join(df1b.alias("b"), col("a.name") == col("b.name"),"left_outer")\
.drop("b.name")

In [68]:
df2.limit(30).toPandas()

,name,states,name,age
0,mary,IN,mary,65
1,mary,IN,mary,9
2,elizabeth,IN,elizabeth,20
3,ryan,KY,ryan,35
4,connor,KY,connor,62
5,connor,KY,connor,68
6,bill,MI,bill,25
7,lisa,PA,lisa,4
8,connor,KY,connor,62
9,connor,KY,connor,68


In [69]:
df2.count()

14

In [10]:
df1p = df1.pivot("states")

AttributeError: 'DataFrame' object has no attribute 'pivot'